In [83]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
from bs4 import BeautifulSoup as bs
import re

In [2]:
#기존의 크롤링 방법 말고 다른 방식을 써보자

id='mugudeli601'
url = "https://www.law.go.kr/DRF/lawSearch.do?OC={}&target=prec&type=XML".format(id)
response = urlopen(url).read()
bs(response)

c:\Users\Heeyo\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


<?xml version="1.0" encoding="UTF-8"?><html><body><precsearch><target>prec</target>&lt;키워드&gt;*키워드&gt;<section>evtNm</section><totalcnt>84941</totalcnt><page>1</page><prec id="1">&lt;판례일련번호&gt;230961판례일련번호&gt;&lt;사건명&gt;&lt;![CDATA[주주총회소집허가]]&gt;사건명&gt;&lt;사건번호&gt;2022마5372사건번호&gt;&lt;선고일자&gt;2022.09.07선고일자&gt;&lt;법원명&gt;대법원법원명&gt;&lt;법원종류코드&gt;법원종류코드&gt;&lt;사건종류명&gt;민사사건종류명&gt;&lt;사건종류코드&gt;400101사건종류코드&gt;&lt;판결유형&gt;결정판결유형&gt;&lt;선고&gt;자선고&gt;&lt;판례상세링크&gt;/DRF/lawService.do?OC=mugudeli601&amp;target=prec&amp;ID=230961&amp;type=HTML&amp;mobileYn=판례상세링크&gt;</prec><prec id="2">&lt;판례일련번호&gt;230971판례일련번호&gt;&lt;사건명&gt;&lt;![CDATA[소유권이전등기]]&gt;사건명&gt;&lt;사건번호&gt;2022다217117사건번호&gt;&lt;선고일자&gt;2022.09.07선고일자&gt;&lt;법원명&gt;대법원법원명&gt;&lt;법원종류코드&gt;법원종류코드&gt;&lt;사건종류명&gt;민사사건종류명&gt;&lt;사건종류코드&gt;400101사건종류코드&gt;&lt;판결유형&gt;판결판결유형&gt;&lt;선고&gt;선고선고&gt;&lt;판례상세링크&gt;/DRF/lawService.do?OC=mugudeli601&amp;target=prec&amp;ID=230971&amp;type=HTML&amp;mobileYn=판례상세링크&gt;</prec><prec id="3">&lt;판례일

In [3]:
xtree = ET.fromstring(response) #response에 저장된 string으로부터 tree를 생성한다

In [24]:
#first check what is in it
'''for child in xtree:
    print(':',child.tag,child.attrib)
print(xtree.find('page').text)'''

"for child in xtree:\n    print(':',child.tag,child.attrib)\nprint(xtree.find('page').text)"

In [4]:
xtree[3].text #totalCnt

'84941'

In [5]:
for node in xtree[5:]:
    print('\n\n',node.attrib)
    tags=[]
    for child in node:
        tags.append(child.tag)
    for t in tags:
        print(node.find(t).text)



 {'id': '1'}
230961
주주총회소집허가
2022마5372
2022.09.07
대법원
None
민사
400101
결정
자
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=230961&type=HTML&mobileYn=


 {'id': '2'}
230971
소유권이전등기
2022다217117
2022.09.07
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=230971&type=HTML&mobileYn=


 {'id': '3'}
230967
폭력행위등처벌에관한법률위반(단체등의구성·활동)·폭력행위등처벌에관한법률위반(단체등의공동공갈)·정보통신망이용촉진및정보보호등에관한법률위반(명예훼손)·아동·청소년의성보호에관한법률위반(성착취물제작·배포등)·폭력행위등처벌에관한법률위반(단체등의공동강요)·폭력행위등처벌에관한법률위반(공동강요)·아동·청소년의성보호에관한법률위반(성착취물소지등)·성폭력범죄의처벌등에관한특례법위반(카메라등이용촬영·반포등)·성폭력범죄의처벌등에관한특례법위반(카메라등이용촬영물소지등)
2022도6993
2022.09.07
대법원
None
형사
400102
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=230967&type=HTML&mobileYn=


 {'id': '4'}
230957
연차수당지급
2022다245419
2022.09.07
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=230957&type=HTML&mobileYn=


 {'id': '5'}
230981
중재판정의집행
2020마5970
2022.09.07
대법원
None
민사
400101
결정
자
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=230981&type=HTML&mobileYn=

In [6]:
totalCnt = int(xtree.find('totalCnt').text)#판례 페이지에 totalCnt라는 요소가 존재하는 것 같다. 
#왜 20개밖에 없나 했는데 원래 한 페이지당 20개라고 한다. 
#페이지마다 loop을 돌아가야 하는 것

In [7]:
#판례 목록 만들기 
cnt=0
page = 1
rows = []
for i in trange(int(totalCnt / 20)): #기다리는 바를 만드는 tqdm의 range 함수인 trange
    if i>1:
        break
    try:
        items = xtree[5:]
    except:
        break
        
    for node in items:
        data={}
        tags=[]
        for child in node:
            tags.append(child.tag)
        for t in tags:
            data[t]=re.sub(',','$',str(node.find(t).text))

        rows.append(data)
    cnt += 1
    url = "https://www.law.go.kr/DRF/lawSearch.do?OC=mugudeli601&target=prec&type=XML&page={}".format(page)
    response = urlopen(url).read()
    xtree = ET.fromstring(response)
    page+=1

  0%|          | 2/4247 [00:00<11:23,  6.21it/s]


In [8]:
#rows에 모든 정보가 담겨있음
prec_list=pd.DataFrame(rows,columns=rows[0].keys())
prec_list.head(5)

,판례일련번호,사건명,사건번호,선고일자,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,선고,판례상세링크
0,230961,주주총회소집허가,2022마5372,2022.09.07,대법원,None,민사,400101,결정,자,/DRF/lawService.do?OC=mugudeli601&target=prec&...
1,230971,소유권이전등기,2022다217117,2022.09.07,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
2,230967,폭력행위등처벌에관한법률위반(단체등의구성·활동)·폭력행위등처벌에관한법률위반(단체등의공...,2022도6993,2022.09.07,대법원,None,형사,400102,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
3,230957,연차수당지급,2022다245419,2022.09.07,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
4,230981,중재판정의집행,2020마5970,2022.09.07,대법원,None,민사,400101,결정,자,/DRF/lawService.do?OC=mugudeli601&target=prec&...


In [9]:
prec_list.columns

Index(['판례일련번호', '사건명', '사건번호', '선고일자', '법원명', '법원종류코드', '사건종류명', '사건종류코드',
       '판결유형', '선고', '판례상세링크'],
      dtype='object')

In [10]:
coluumns_new={'판례일련번호':'datanumber', '사건명':'title', '사건번호':'casenumber', '선고일자':'date', '법원명':'court', '법원종류코드':'courttype', '사건종류명':'casetypename', '사건종류코드':'casecode',
       '판결유형':'verdicttype', '선고':'sentence', '판례상세링크':'detaillink'}
prec_list=prec_list.rename(columns=coluumns_new)

In [11]:
#prec_list를 traverse하며 모든 판례정보를 긁기
import re

contents = ['사건명', '사건번호', '선고일자','판결유형','판시사항', '판결요지', '참조조문', '참조판례', '판례내용']
contents_rename=['title','casenumber','date','verdicttype','holding','summary','jmreference','prreference','prny']
rename_dict={a:b for a,b in zip(contents,contents_rename)}

In [12]:
rename_dict

{'사건명': 'title',
 '사건번호': 'casenumber',
 '선고일자': 'date',
 '판결유형': 'verdicttype',
 '판시사항': 'holding',
 '판결요지': 'summary',
 '참조조문': 'jmreference',
 '참조판례': 'prreference',
 '판례내용': 'prny'}

In [39]:
prec_list.columns

Index(['datanumber', 'title', 'casenumber', 'date', 'court', 'courttype',
       'casetypename', 'casecode', 'verdicttype', 'sentence', 'detaillink'],
      dtype='object')

In [13]:


def remove_tag(content):
    cleaned=re.sub(',','$',re.sub('<.*?>', '', content))
    return cleaned

case_list=[]
for pid,l in zip(prec_list['datanumber'],prec_list['detaillink']):
    url="https://www.law.go.kr"+l.replace('HTML','XML')
    response=urlopen(url).read()
    xtree=ET.fromstring(response)
    row=[]
    row.append(pid)
    for content in contents:
        try:
            text=xtree.find(content).text
            text=remove_tag(text)
        except:
            text="-"
        row.append(text)
    case_list.append(row)
    
case_list=pd.DataFrame(case_list,columns=['datanumber']+contents)
case_list.reset_index(inplace=True)
case_list=case_list.rename(columns=rename_dict)        

In [27]:
len(case_list)
print(case_list['casenumber'].value_counts())

2022마5372      1
2022다217117    1
2021다311111    1
2020도16897     1
2018다212610    1
2019도7370      1
2020후11479     1
2020도1007      1
2017두53170     1
2021다216766    1
2019다282050    1
2018다212740    1
2018다304014    1
2022다244805    1
2022도8341      1
2022다237098    1
2020마5970      1
2022다245419    1
2022도6993      1
2019두58773     1
Name: casenumber, dtype: int64


In [20]:
case_list=case_list.drop_duplicates(['datanumber'])

In [22]:
from pathlib import Path  
filepath = Path('D:\\PanryeAI\\case_list.csv')  
case_list.to_csv(filepath,index=False)

### kobert 파인튜닝 시도

In [86]:
#checking if title has the ability to be a class definition for finetuning Kobert
from konlpy.tag import Kkma
from sklearn.cluster import KMeans
from gensim.models.word2vec  import Word2Vec 
from gensim.models.doc2vec import Doc2Vec

model=Doc2Vec.load('doc2vec.model')
prec_list['sentence'].value_counts()
prec_list['title'].value_counts()

def map_and_combine(l):
  ret=''
  for w in l:
    ret+=' '+w
  return ret[1:]

kkma=Kkma()

def host(sent):
    global model 
    return model.infer_vector(sent)
  
def host_w(word):
  model=Word2Vec.load('word2vec.model')
  
  try:
    return model.wv[word]
  except:
    Exception('wv not returned') 
  
def replace_w(word):
  model=Word2Vec.load('word2vec.model')
  
  try:
    return model.wv.most_similar(word,topn=1)
  except:
    return 
    

sents=[]
for title in prec_list['title'].values:
    sents.append(map_and_combine(kkma.nouns(title)))


In [62]:
print(sents)

['주주 주주총회소집허가 총회 소집 허가', '소유권 소유권이전등기 이전 등기', '등 폭력 등처벌 폭력행위 처벌 행위 법률 법률위반 위반 단체 단체등 구성 활동 공동 공동공갈 공갈 정보 촉진 정보보호 정보통신망이용 보호 통신망 이용 명예 명예훼손 훼손 아동 청소년 청소년의성보호 의성 성 성착취물제작 착취 물 제작 배포 배포등 공동강요 강요 성착취물소지 소지 성폭력 처벌등 성폭력범죄 범죄 특례법 특례법위반 카메라 카메라등이용촬영 촬영 반포 반포등 등이용촬영 물소지등', '연차 연차수당지급 수당 지급', '중 중재판정의집행 재판 정의 집행', '손해 손해배상 배상 기', '등 특정 특정범죄가중처벌 범죄 가중 처벌 법률 법률위반 위반 향 향정 정 마약 관리 대마', '공유물 공유물분할 분할', '보험금', '구상', '건축 건축주명 주명 변경 변경절차이행청구 절차 이행 청구', '소유권 소유권이전등기 이전 등기', '부가 부가가치세경정거부 가치세 경정 거부 처분 처분취소 취소', '통신 통신비밀보호법위반 비밀 보호법 위반', '거절 거절결정 결정 특', '모욕', '손해 손해배상 배상 기 긴급 긴급조치 조치 제 제9호위반 9 호 위반 혐의 수사 유죄 유죄판결 판결 사람 유족 대통령 수사기관 기관 법원 불법 불법행위 행위 이유 국가 국가배상 청구 사안', '모욕', '청구 청구이의 이의', '하수도 하수도원인자부담금부과 원인자 부담금 부과 처분 처분취소 취소', '주주 주주총회소집허가 총회 소집 허가', '소유권 소유권이전등기 이전 등기', '등 폭력 등처벌 폭력행위 처벌 행위 법률 법률위반 위반 단체 단체등 구성 활동 공동 공동공갈 공갈 정보 촉진 정보보호 정보통신망이용 보호 통신망 이용 명예 명예훼손 훼손 아동 청소년 청소년의성보호 의성 성 성착취물제작 착취 물 제작 배포 배포등 공동강요 강요 성착취물소지 소지 성폭력 처벌등 성폭력범죄 범죄 특례법 특례법위반 카메라 카메라등이용촬영 촬영 반포 반포등 등이용촬영 물소지등', '연차 연차수당지급 수당 지급', '중 

In [88]:
sentvecs2=[]
def excludeNone(li):
    newli=[]
    for i in li:
        if i != None:
            newli.append(i)
    return newli
print('test',excludeNone([None,1,'2']))

sent2vec_wv=[]
for sent in sents:
    tmpsent=[]
    count=0
    for i in list(map(replace_w, sent.split())):
        try:
            tmpsent.append(i[0][0])
            count+=1
        except:
            continue
    print(tmpsent)
    wvlist=list(map(host_w,tmpsent))
    sv=np.array(wvlist)
    sent2vec_wv.append(sv/count)
entirevec=host(sents)

test [1, '2']
['주식', '사회', '종중', '신용']
AHHHHHHHHhh
 [array([ 0.04629849,  0.15435731,  0.36360717, -0.18629965, -0.03142688,
       -0.3850127 ,  0.1557931 ,  0.4881797 , -0.0832911 , -0.06341872,
       -0.304162  , -0.43297866, -0.07251558,  0.0969827 , -0.00834098,
       -0.18378982,  0.00060405, -0.2692818 ,  0.2727749 , -0.3988429 ,
        0.24378873,  0.07724002, -0.23353972, -0.11578249, -0.1725535 ,
       -0.13352515, -0.16243461, -0.26709697, -0.23266156, -0.03771424,
        0.34911397,  0.16477813, -0.34607953, -0.19173382, -0.13088761,
        0.0237998 , -0.1167518 , -0.32324252, -0.11980835, -0.52016747,
        0.05557444, -0.22380319,  0.00368889, -0.10154627,  0.14373064,
       -0.03745428, -0.1191423 , -0.1529187 ,  0.02458164,  0.06937773,
        0.2322906 , -0.09285446, -0.09545343, -0.17818125,  0.2092387 ,
        0.15817378,  0.23403423,  0.04459861, -0.28586984,  0.09996777,
        0.15169103,  0.2921922 , -0.01931693, -0.19396427, -0.18837944,
        0.2

In [80]:
sentvecs.shape

(100,)

In [50]:

y_pred = KMeans(n_clusters=5).fit_predict(sentvecs)

ValueError: Expected 2D array, got 1D array instead:
array=[-1.20992407e-01 -3.84119749e-02  1.87043678e-02  1.86018571e-01
 -1.31134763e-01 -1.11821242e-01  1.57989915e-02  1.89967543e-01
  8.00215453e-02 -2.53690276e-02  4.10149358e-02 -8.51599574e-02
 -5.41429296e-02  1.95363946e-02  8.92506167e-02 -1.13117270e-01
 -1.11835361e-01 -3.07809561e-02  1.05492272e-05  4.97742817e-02
  1.62878856e-01  8.10653120e-02 -5.28755859e-02  1.89448133e-01
 -4.84228581e-02 -3.14365663e-02 -1.00026675e-01  1.24986898e-02
 -1.07856113e-02 -1.71199664e-01  4.40783203e-02 -2.75486615e-04
  8.88585448e-02  5.11313900e-02 -1.15993537e-01  1.40602496e-02
  9.08433571e-02 -1.33977495e-02 -1.41478166e-01 -1.53993487e-01
  1.24131637e-02  5.84173240e-02 -7.78411776e-02 -1.67454809e-01
 -9.06569511e-02  3.70708876e-03 -8.20517987e-02  1.64951369e-01
  4.08039279e-02  1.49457417e-02 -1.12336382e-01  8.47936198e-02
  2.72572171e-02 -5.83867282e-02 -4.85791974e-02  3.29111032e-02
  2.68766051e-03  8.03969353e-0

In [45]:
case_list.head(10)

,index,datanumber,title,casenumber,date,verdicttype,holding,summary,jmreference,prreference,prny
0,0,230881,소유권이전등기,2021다216766,20220831,판결,[1] 1975. 12. 31. 지적법 개정 전에 복구된 구 토지대장이나 분배농...,[1] 1975. 12. 31. 지적법 개정 전에 복구된 구 토지대장상의 소유자...,[1] 민법 제186조$ 민사소송법 제202조$ 구 지적법(1975. 12. 31...,[1] 대법원 2013. 6. 27. 선고 2012다91354 판결$ 대법원 20...,【원고$ 피상고인】 원고【피 고】 피고【피고보조참가인$ 상고인】 피고보조참가인...
1,1,230867,보험금,2018다304014,20220831,판결,甲 자산운용회사가 乙 보험회사와 체결한 자산운용전문인 배상책임보험계약의 계약서 ...,甲 자산운용회사가 乙 보험회사와 체결한 자산운용전문인 배상책임보험계약의 계약서 ...,민법 제105조,-,【원고$ 피상고인】 칸서스자산운용 주식회사 (소송대리인 법무법인(유한) 지평 담당변...
2,2,230871,통신비밀보호법위반,2020도1007,20220831,판결,통신비밀보호법 제14조 제1항의 금지를 위반하는 행위는 같은 법 제3조 제1항 위반...,통신비밀보호법은 공개되지 않은 타인 간의 대화에 관하여 다음과 같이 정하고 있다...,통신비밀보호법 제3조 제1항$ 제14조 제1항$ 제16조 제1항,대법원 2006. 10. 12. 선고 2006도4981 판결(공2006하$ 1939...,【피 고 인】 피고인【상 고 인】 피고인【원심판결】 부산고법 2020. 1. 9. ...
3,3,230873,부가가치세경정거부처분취소,2017두53170,20220831,판결,어떠한 금액이 부가가치세의 과세표준에 포함되지 않는 에누리액에 해당하기 위해서는...,구 부가가치세법(2013. 6. 7. 법률 제11873호로 전부 개정되기 전의 것)...,구 부가가치세법(2013. 6. 7. 법률 제11873호로 전부 개정되기 전의 것)...,-,【원고$ 상고인】 에스케이텔레콤 주식회사 (소송대리인 법무법인(유한) 태평양 외 2...
4,4,230869,구상금,2018다212740,20220831,판결,[1] 자동차상해보험계약에 따른 보험금을 지급한 보험자가 보험자대위를 할 수 있...,[1] 자동차상해보험은 그 성질상 상해보험에 속하므로$ 자동차상해보험계약에 따른...,[1] 상법 제729조 / [2] 상법 제729조,[1] 대법원 2004. 7. 9. 선고 2003다29463 판결(공2004하$ ...,【원고$ 상고인 겸 피상고인】 삼성화재해상보험 주식회사 (소송대리인 법무법인(유한)...
5,5,230877,모욕,2019도7370,20220831,판결,[1] 모욕죄의 보호법익(=외부적 명예) 및 ‘모욕’의 의미 / 어떠한 표현이 ...,[1] 형법 제311조 모욕죄는 사람의 인격적 가치에 대한 사회적 평가를 의미하...,[1] 헌법 제21조$ 형법 제311조 / [2] 헌법 제21조$ 형법 제311조,[1] 대법원 2018. 11. 29. 선고 2017도2661 판결(공2019상$...,【피 고 인】 피고인【상 고 인】 피고인【원심판결】 수원지법 2019. 5. 3. ...
6,6,230879,건축주명의변경절차이행청구의소,2019다282050,20220831,판결,건축허가 또는 신고에 관한 건축주 명의가 수인으로 되어 있을 경우$ 공동건축주 ...,행정청으로부터 허가를 받거나 행정청에 신고(이하 이러한 허가와 신고를 합하여 ‘...,민법 제265조,대법원 2015. 9. 10. 선고 2012다23863 판결(공2015하$ 1457),【원고$ 상고인】 대한예수교장로회 ○○교회 (소송대리인 법무법인 다비다 담당변호사 ...
7,7,230859,손해배상(기)[긴급조치 제9호위반 혐의로 수사 및 유죄판결을 받은 사람들 또는 그 ...,2018다212610,20220830,전원합의체 판결,구 국가안전과 공공질서의 수호를 위한 대통령긴급조치(긴급조치 제9호)의 발령·적...,[다수의견] 보통 일반의 공무원을 표준으로 공무원이 직무를 집행하면서 객관적 주...,국가배상법 제2조 제1항$ 헌법 제29조$ 구 헌법(1980. 10. 27. 헌법 ...,대법원 2000. 5. 12. 선고 99다70600 판결(공2000하$ 1403)$...,【원고$ 상고인】 별지 원고 명단 기재와 같다. (소송대리인 법무법인 덕수 담당변호...
8,8,230901,법인세부과처분취소,2017두41313,20220825,판결,[1] 구 국세기본법 제14조 제3항을 적용하여 거래 등의 실질에 따라 과세하기...,[1] 구 국세기본법(2010. 1. 1. 법률 제9911호로 개정되기 전의 것...,[1] 구 국세기본법(2010. 1. 1. 법률 제9911호로 개정되기 전의 것)...,-,【원고$ 상고인】 주식회사 장미트레이딩 (소송대리인 법무법인(유한) 율촌 외 1인)...
9,9,230863,손해배상(기),2019다229202,20220825,전원합의체 판결,[1] 채권자대위권의 행사요건인 ‘보전의 필요성’을 인정하기 위한 요건 및 이를...,[1] 채권자는 자기의 채권을 보전하기 위하여 일신에 전속한 권리가 아닌 한 채...,[1] 민법 제404조 제1항 / [2] 민법 제404조 제1항,[1] 대법원 2020. 5. 21. 선고 2018다879 전원합의체 판결(공20...,【원고$ 피상고인】 삼성화재해상보험 주식회사 (소송대리인 법무법인 도원 외 1인)【...
